In [ ]:
import sys
sys.path.append("./src")
import os
import datetime
import pandas as pd
import numpy as np

from getPipeline import data_all_pipeline
from sindy_utils import library_size
from training import train_network
import tensorflow as tf

In [ ]:
params = {}

params['input_dim'] = 1000
params['latent_dim'] = 20
params['model_order'] = 2
params['poly_order'] = 3
params['include_sine'] = True
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.0
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'elu'
params['widths'] = [128,64,32]

# training parameters
params['epoch_size'] = 1000
params['batch_size'] = 32
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/results'
params['print_progress'] = True
params['print_frequency'] = 10

# training time cutoffs
params['max_epochs'] = 50
params['refinement_epochs'] = 10

In [ ]:
training_data, validation_data = data_all_pipeline(window_size=1000, step_size=1000, val_split=0.1)

In [ ]:
num_experiments = 1
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'Signal_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()
    
    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')